In [1]:
from torch.utils.data import DataLoader, random_split
from torch.optim import Adam, RMSprop
from torch.optim.lr_scheduler import OneCycleLR, StepLR, LinearLR
from data.preprocessing import *
from data.data_utils import *
from models.mult2multatt import *
from utils import *

In [23]:
hidden_size_fc = 150
hidden_size_mg = 150
architecture = (100,50,25,5)
num_layers = 5
sequence_length_hour = 96  #last 5 days
sequence_length_minute = 300 #minute
output_size = 1 #depends on target
pred_length = 1 #hours
dict_values = ['dst_kyoto', 'kp_gfz']
time_steps = 0 #after how time steps you want to inference

In [24]:
device = get_default_device()

In [25]:
device

device(type='cuda')

In [27]:
arch = 'LSTM'
encoder_fc = EncoderMultiheadAttentionLSTM(input_size = 9,hidden_size =  hidden_size_fc, num_heads=3, num_layers = num_layers, bidirectional=False)
encoder_mg = EncoderMultiheadAttentionLSTM(input_size = 11, hidden_size = hidden_size_mg, num_heads=11, num_layers = num_layers, bidirectional=False)
decoder = DecoderMultiheadAttentionLSTM(input_size = encoder_mg.hidden_size+encoder_fc.hidden_size, hidden_size = hidden_size_fc+hidden_size_mg, num_heads = 20, num_layers = num_layers, bidirectional=False)
model = to_device(MultiHeaded2MultiheadAttentionLSTM(encoder_fc, encoder_mg, decoder, [30,30], architecture, output_size), device)

In [28]:
#DATA PROCESSING
start_time = '20210101'
end_time = '20230802'
scrap_date = interval_time(start_time, end_time)
months = list(set([day[:6] for day in scrap_date]))
import_Dst(months)
l1_sample, l2_sample, dst, kp = automated_preprocessing(scrap_date, sep = True)
l1_sample_hour = (l1_sample[0].resample('60min').mean(), l1_sample[1].resample('60min').mean()) #multhead
l2_sample_hour = (l2_sample[0].resample('60min').mean(), l2_sample[1].resample('60min').mean()) #multhead encoder forcing

c:\Users\jorge\Documents\SMFGF-SpaceApps\data\preprocessing.py:360: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  today_kp = kp[day][0:8]


In [29]:
hour_kp_dataset = MainToSingleTarget(l1_sample_hour, dst, sequence_length_hour, pred_length, True, True, 'dst_kyoto', None, time_step_ahead = time_steps)

In [30]:
#Test:20% training: 80%

test_size = round(0.2*len(hour_kp_dataset))

train_hour_kp, test_hour_kp = random_split(hour_kp_dataset , [len(hour_kp_dataset) - test_size, test_size])

batch_size = 100  #Change based on GPU capacity

train_hour_kp_dl = DataLoader(train_hour_kp, batch_size, shuffle=True, num_workers=4, pin_memory=True)
train_hour_kp_dl = DeviceDataLoader(train_hour_kp_dl, device)
test_hour_kp_dl = DataLoader(test_hour_kp, batch_size*2, num_workers=4, pin_memory=True)
test_hour_kp_dl = DeviceDataLoader(test_hour_kp_dl, device)

In [31]:
history = []

In [34]:
##hyperparameters
epochs = 100
max_lr =0.00045
weigth_decay = 1e-6
grad_clip = 0
opt_func = Adam
#lr_sched = OneCycleLR  
lr_sched = LinearLR
start_factor = 1
end_factor = 0.1
steps = epochs
gamma = 0.999
weights = [0.1,0.1,0.2,1]
encoder_forcing = False
#opt_func = RMSprop

In [35]:
history += model.fit(epochs, max_lr, train_hour_kp_dl, test_hour_kp_dl, weigth_decay, grad_clip, opt_func, lr_sched, start_factor, end_factor, steps, gamma, weights, encoder_forcing)

Epoch [0]:
	last_lr: 0.00050
	train_loss: 0.1995
	val_loss: 0.1612
	r2_score: 0.8224
	precision: nan
	recall: 0.5808
	f1_score: nan
Epoch [1]:
	last_lr: 0.00050
	train_loss: 0.1604
	val_loss: 0.1611
	r2_score: 0.8224
	precision: nan
	recall: 0.6730
	f1_score: nan
Epoch [2]:
	last_lr: 0.00049
	train_loss: 0.1433
	val_loss: 0.1432
	r2_score: 0.8424
	precision: nan
	recall: 0.6900
	f1_score: nan
Epoch [3]:
	last_lr: 0.00049
	train_loss: 0.1350
	val_loss: 0.1266
	r2_score: 0.8600
	precision: nan
	recall: 0.7359
	f1_score: nan
Epoch [4]:
	last_lr: 0.00048
	train_loss: 0.1188
	val_loss: 0.1209
	r2_score: 0.8663
	precision: 0.7578
	recall: 0.8785
	f1_score: nan
Epoch [5]:
	last_lr: 0.00048
	train_loss: 0.1108
	val_loss: 0.1013
	r2_score: 0.8876
	precision: nan
	recall: 0.7834
	f1_score: nan
Epoch [6]:
	last_lr: 0.00047
	train_loss: 0.0957
	val_loss: 0.1212
	r2_score: 0.8655
	precision: 0.7352
	recall: 0.8628
	f1_score: nan
Epoch [7]:
	last_lr: 0.00047
	train_loss: 0.0870
	val_loss: 0.1001
	r2

KeyboardInterrupt: 

In [36]:
import os
os.makedirs('mult2mult', exist_ok=True)
torch.save(model, f'mult2mult/{hidden_size_fc}_{hidden_size_mg}_{num_layers}_{encoder_forcing}_{time_steps}_{arch}.pt')
try:     
    os.remove(f'mult2mult/{hidden_size_fc}_{hidden_size_mg}_{num_layers}_{encoder_forcing}_{time_steps}_{arch}.csv')
except FileNotFoundError:   
    pass
with open(f'mult2mult/{hidden_size_fc}_{hidden_size_mg}_{num_layers}_{encoder_forcing}_{time_steps}_{arch}.csv', 'w') as file:
    file.write(','.join(map(str,list(history[-1].values()))))

IndexError: list index out of range